In [40]:
:opt no-lint

In [41]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

# 17 Applicative
## 17.1 Applicative
Applicatives are monoidal functors. They allows for function application lifted over structure (like `Functor`), but the function is also embedded into a structure, hence the two structures must be combined.
## 17.2 Defining Applicative
An `Applicative` instance must also have a `Functor` instance.
The `pure` function lifts something into functorial (applicative) structure.
The `<*>` infix operator is similar to `fmap` but the functorial structure is also around the applied function.
Moreover, the `Control.Applicative` library provides other convenient functions: `liftA` (basically the same as `fmap`), `liftA2`, and `liftA3`.
## 17.3 Functor vs. Applicative
Defining a `Functor` in terms of a provided `Applicative` instance:

In [42]:
fmap' f x = pure f <*> x

## 17.4 Applicative functors are monoidal functors
Let's compare:

In [43]:
:t ($)
:t (<$>)
:t (<*>)
:t mappend

($) :: forall a b. (a -> b) -> a -> b

(<$>) :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

(<*>) :: forall (f :: * -> *) a b. Applicative f => f (a -> b) -> f a -> f b

mappend :: forall a. Monoid a => a -> a -> a

When `f` is a type with a `Monoid` instance, we have a `Monoid` for our structure and function application for our values.
Example:

In [44]:
("Woo", (+1)) <*> (" Hoo!", 0)

("Woo Hoo!",1)

the monoidal bit may not be easily recognizable as the canonical `mappend` of that type, because some types can have multiple monoids.
## 17.5 Applicative in use
The `[]` instance behaves as follow:
* application: each function of the left list is applied to each element of the right list
* structure: all the resulting list are concatenated using the `Monoid` instance of `[]`
For `Maybe` instance compare `Just f <*> Just a = Just (f a)` with `mappend (Just a) (Just a') = Just (mappend a a')`
#### Exercises: Lookups
In the following exercises, you will need to use the given terms to make the expressions type check:
1. `pure`
2. `(<$>)`
3. `(<*>)`

Make the following expressions type check:
1. 

In [45]:
added :: Maybe Integer
added = (+3) <$> (lookup 3 $ zip [1, 2, 3] [4, 5, 6])

2. 

In [46]:
y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

3. 

In [47]:
import Data.List (elemIndex)

x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = max' <$> x <*> y

4. 

In [48]:
xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = sum <$> ((,) <$> x <*> y)

### Identity
The `Identity` type introduces structure without changing the semantics.
#### Exercise: Identity instance
Write an `Applicative` instance for `Identity`:

In [49]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
  fmap f (Identity x) = Identity $ f x

instance Applicative Identity where
  pure = Identity
  (Identity f) <*> (Identity x) = Identity $ f x

### Constant
It provides a strucure and also acts like the `const` function (remove function application)
#### Exercise: Constant instance
Write an `Applicative` instance for `Constant`:

In [50]:
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Ord, Show)

instance Functor (Constant a) where
  fmap _ = Constant . getConstant 

instance Monoid a => Applicative (Constant a) where
  pure _ = Constant mempty
  (Constant _) <*> (Constant x) = Constant x

### Maybe Applicative
The `Maybe` applicative can be used to build a type that must be a `Just` only when all its parameters are `Just`.
#### Exercise: Fixer upper
Given the functions and values provided, use `<$>`, `<*>` and `pure` to fill in missing bits of the broken code below to make it work:
1. 

In [51]:
const <$> Just "Hello" <*> pure "World"

Just "Hello"

2. 

In [52]:
(,,,) <$> Just 90 <*> Just 10 <*> Just "Tierness" <*> pure [1, 2, 3]

Just (90,10,"Tierness",[1,2,3])

## 17.6 Applicative laws
1. Identity: `pure id <*> v = v`
2. Composition: `pure (.) <*> u <*> v <*> w = u <*> (v <*> w)`
3. Homomorphism (structure-preserving map between two algebraic structures): `pure f <*> pure x = pure (f x)`
This law is enabled by the `Monoid` constraint. The general idea of the homomorphism law is that applying the function doesn’t change the structure around the values.
4. Interchange: `u <*> pure y = pure ($ y) <*> u`

## 17.7 You knew this was coming
We can test the properties with `checkers` library:

In [53]:
xs = [("b", "w", 1)]
quickBatch $ applicative xs


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

## 17.8 `ZipList` Monoid
The default monoid of lists in the GHC `Prelude` is concatenation. An alternative monoid is `ZipList` that combines the values of the two lists as parallel sequences using a monoid provided by the values themselves.
### Zero vs. Identity
Implementation of `pure` must satisfy identity `mempty = pure mempty` into `Monoid` instance of `ZipList`.
#### List Applicative exercise
Implement `Applicative` for lists. Use the `checkers` library to validate the instance:

In [54]:
data List a = Nil | Cons a (List a) deriving (Eq, Show)

instance Functor List where
  fmap _ Nil = Nil
  fmap f (Cons x xs) = Cons (f x) (fmap f xs)

instance Applicative List where
  pure = flip Cons Nil
  fs <*> xs = flatMap (<$> xs) fs
  -- below the previous implementation without flatMap
  -- Nil <*> _ = Nil
  -- (Cons f fs) <*> xs = append (f <$> xs) (fs <*> xs)
  
append :: List a -> List a -> List a
append Nil ys = ys
append (Cons x xs) ys = Cons x $ xs `append` ys

fold :: (a -> b -> b) -> b -> List a -> b
fold _ b Nil = b
fold f b (Cons h t) = f h (fold f b t)

concat' :: List (List a) -> List a
concat' = fold append Nil

-- write this one in terms of concat' and fmap
flatMap :: (a -> List b) -> List a -> List b
flatMap f  = concat' . fmap f 

instance Arbitrary a => Arbitrary (List a) where
  arbitrary = do
      x <- arbitrary
      pure $ Cons x Nil
      
instance Eq a => EqProp (List a) where
  xs =-= ys = xs' `eq` ys'
    where
      xs' = let l = xs in take' 100 l
      ys' = let l = ys in take' 100 l

take' :: Int -> List a -> List a
take' 0 _ = Nil
take' _ Nil = Nil
take' n (Cons x xs) = Cons x (take' (n - 1) xs)

x = ("1", "2", "3")
quickBatch $ applicative (Cons x Nil)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

`Applicative`, unlike `Functor`, is not guaranteed to have a unique implementation for a given datatype.
#### `ZipList` Applicative exercise
Implement the `ZipList` Applicative. Use the `checkers` library to validate the instance.

In [55]:
newtype ZipList' a = ZipList' (List a) deriving (Eq, Show)
      
instance Functor ZipList' where
  fmap f (ZipList' xs) = ZipList' $ fmap f xs
  
instance Applicative ZipList' where
  pure = ZipList' . pure
  (ZipList' fs) <*> (ZipList' xs) = ZipList' $ zipList fs xs
    where
      zipList _   Nil = Nil
      zipList Nil _   = Nil
      zipList (Cons f Nil) (Cons x xs)  = Cons (f x) (pure f <*> xs)
      zipList (Cons f fs)  (Cons x Nil) = Cons (f x) (fs <*> pure x)
      zipList (Cons f fs)  (Cons x xs)  = Cons (f x) (zipList fs xs)

instance Arbitrary a => Arbitrary (ZipList' a) where
  arbitrary = ZipList' <$> arbitrary

instance Eq a => EqProp (ZipList' a) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (ZipList' (Cons x Nil))


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

### Either and Validation Applicative
### Either vs. Validation
`Validation` datatype is identical to `Either` but the `Applicative` instance differs as it combines the errors using the `Monoid` instance instead of the shortcut semantic.
#### Exercise: Variations on Either
Implement the `Validation` Applicative. Use the `checkers` library to validate the instance.

In [56]:
data Validation e a = Failure e | Success a deriving (Eq, Show)

-- same as Either
instance Functor (Validation e) where
  fmap _ (Failure x) = Failure x
  fmap f (Success x) = Success $ f x

-- This is different
instance Monoid e => Applicative (Validation e) where
  pure = Success
  (Success f) <*> (Success x) = Success $ f x
  (Success _) <*> (Failure e) = Failure e
  (Failure e) <*> (Failure e') = Failure $ mappend e e'
  (Failure e) <*> (Success _) = Failure e

instance (Arbitrary e, Arbitrary a) => Arbitrary (Validation e a) where
  arbitrary = do
    f <- arbitrary
    x <- arbitrary
    elements [Failure f, Success x]
      
instance (Eq e, Eq a)  => EqProp (Validation e a) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Success x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

## 17.9 Chapter exercises
Given a type that has an instance of Applicative, specialize the types
of the methods. Test your specialization in the REPL. One way to
do this is to bind aliases of the type class methods to more concrete
types that have the type we tell you to fill in:
1. `[]`

In [57]:
:set -XTypeApplications

:type pure @[]
:type (<*>) @[]

pure @[] :: forall a. a -> [a]

(<*>) @[] :: forall a b. [a -> b] -> [a] -> [b]

2. `IO`

In [58]:
:type pure @IO
:type (<*>) @IO

pure @IO :: forall a. a -> IO a

(<*>) @IO :: forall a b. IO (a -> b) -> IO a -> IO b

3. `(,) a`

In [59]:
:type pure @((,) String)
:type (<*>) @((,) String)

pure @((,) String) :: forall a. a -> (String, a)

(<*>) @((,) String) :: forall a b. (String, a -> b) -> (String, a) -> (String, b)

4. `(->) e`

In [60]:
:type pure @((->) String)
:type (<*>) @((->) String)

pure @((->) String) :: forall a. a -> String -> a

(<*>) @((->) String) :: forall a b. (String -> a -> b) -> (String -> a) -> String -> b

Write instances for the following datatypes. Use the `checkers` library to validate the instances:

1. 

In [61]:
data Pair a = Pair a a deriving (Eq, Show)

instance Functor Pair where
  fmap f (Pair x x') = Pair (f x) (f x')

instance Applicative Pair where
  pure x = Pair x x
  (Pair f f') <*> (Pair x x') = Pair (f x) (f' x')

instance Arbitrary a => Arbitrary (Pair a) where
  arbitrary = do
    x <- arbitrary
    pure $ Pair x x

instance Eq a => EqProp (Pair a) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Pair x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

2. This should look familiar:

In [62]:
data Two a b = Two a b deriving (Eq, Show)

instance Functor (Two a) where
  fmap f (Two x x') = Two x (f x')
  
instance Monoid a => Applicative (Two a) where
  pure = Two mempty
  (Two x f) <*> (Two x' y) = Two (x <> x') (f y)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Two a b) where
  arbitrary = Two <$> arbitrary <*> arbitrary
      
instance (Eq a, Eq b) => EqProp (Two a b) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Two x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

3. 

In [63]:
data Three a b c = Three a b c deriving (Eq, Show)

instance Functor (Three a b) where
  fmap f (Three x y z) = Three x y (f z)
  
instance (Monoid a, Monoid b) => Applicative (Three a b) where
  pure = Three mempty mempty
  (Three x z f) <*> (Three x' z' y) = Three (x <> x') (z <> z') (f y)

instance (Arbitrary a, Arbitrary b, Arbitrary c) => Arbitrary (Three a b c) where
  arbitrary = Three <$> arbitrary <*> arbitrary <*> arbitrary
      
instance (Eq a, Eq b, Eq c) => EqProp (Three a b c) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Three x x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

4. 

In [64]:
data Three' a b = Three' a b b deriving (Eq, Show)

instance Functor (Three' a) where
  fmap f (Three' x y y') = Three' x (f y) (f y')
  
instance Monoid a => Applicative (Three' a) where
  pure x = Three' mempty x x
  (Three' x f g) <*> (Three' x' y y') = Three' (x <> x') (f y) (g y')

instance (Arbitrary a, Arbitrary b) => Arbitrary (Three' a b) where
  arbitrary = do
    x <- arbitrary
    y <- arbitrary
    pure $ Three' x y y
      
instance (Eq a, Eq b) => EqProp (Three' a b) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Three' x x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

5. 

In [65]:
data Four a b c d = Four a b c d deriving (Eq, Show)

instance Functor (Four a b c) where
  fmap f (Four x y z t) = Four x y z (f t)
  
instance (Monoid a, Monoid b, Monoid c) => Applicative (Four a b c) where
  pure = Four mempty mempty mempty
  (Four x y z f) <*> (Four x' y' z' t) = Four (x <> x') (y <> y') (z <> z') (f t)

instance (Arbitrary a, Arbitrary b, Arbitrary c, Arbitrary d) => Arbitrary (Four a b c d) where
  arbitrary = Four <$> arbitrary <*> arbitrary <*> arbitrary <*> arbitrary
      
instance (Eq a, Eq b, Eq c, Eq d) => EqProp (Four a b c d) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Four x x x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

6. 

In [66]:
data Four' a b = Four' a a a b deriving (Eq, Show)

instance Functor (Four' a) where
  fmap f (Four' x y z t) = Four' x y z (f t)
  
instance Monoid a => Applicative (Four' a) where
  pure = Four' mempty mempty mempty
  (Four' x y z f) <*> (Four' x' y' z' t) = Four' (x <> x') (y <> y') (z <> z') (f t)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Four' a b) where
  arbitrary = do
    x <- arbitrary
    y <- arbitrary
    pure $ Four' x x x y
      
instance (Eq a, Eq b) => EqProp (Four' a b) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ applicative (Four' x x x x)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

### Combinations
Write a function to generate all the possible combinations of three input lists, using `liftA3` from `Control.Applicative`:

In [67]:
import Control.Applicative (liftA3)

stops :: String
stops = "pbtdkg"

vowels :: String
vowels = "aeiou"

combos :: [a] -> [b] -> [c] -> [(a, b, c)]
combos = liftA3 (,,)

## 17.10 Definitions
1. `Applicative` can be thought of as characterizing monoidal functors in Haskell. For a Haskeller’s purposes, it’s a way to functorially apply a function that is embedded in a structure f of the same type as the value you’re mapping it over:

`fmap  ::   (a -> b) -> f a -> f b`

`(<*>) :: f (a -> b) -> f a -> f b`